In [1]:
import pandas as pd
import numpy as np
import glob 
import matplotlib.pyplot as plt
import urllib
import zipfile
import os
from scipy.fft import rfft, rfftfreq

In [3]:
def dataReader(path_names):
    data = []
    for i in path_names:
        data_file = pd.read_csv(i,header=None)
        data.append(data_file)
    return np.array(data)

In [4]:
target_rate = 500
ORIGINAL_SAMPLE_RATE = 50000 # taxa de amostragem original dos dados

In [5]:
def apply_rfft(signal, sampling_rate):
    """
    Aplica a Transformada Rápida de Fourier Real (RFFT) a um sinal de entrada.

    Parâmetros:
    signal (numpy array): O sinal de entrada.
    sampling_rate (float): A taxa de amostragem do sinal.

    Retorna:
    tuple: Frequências e coeficientes da RFFT.
    """
    # Aplicar RFFT
    rfft_result = np.abs(np.fft.rfft(signal, axis=0))[1:,:] #/ signal.shape[0]
    rfft_freqs = np.fft.rfftfreq(len(signal), d=1/sampling_rate)
    
    # spectrum = np.fft.rfftn(data)
    
#     # Calcular as frequências correspondentes
#     freqs = np.fft.rfftfreq(len(data), d=1/sample_rate)
#     # freqs = [np.fft.fftfreq(n, d=1/sample_rate) for n in data.shape]

    return rfft_result, rfft_freqs 

In [6]:
def resample(data, sample_target_rate):
    data_resampled = pd.DataFrame()
    period = 5 # seconds
    step = int(len(data)/period / sample_target_rate )
    start_index = 0
    stop_index = step
    for i in range(0, len(data), step):
        # data_resampled = pd.concat([data_resampled, data[start_index:stop_index].mean().to_frame().T], axis=0, ignore_index=True) ## para funcionar com dataframe
        temp_df =  pd.DataFrame([data[start_index:stop_index].mean(axis=0)])
        # data_resampled = pd.concat([data_resampled, temp_df], ignore_index=True, axis=0) # para ndarray
        data_resampled = pd.concat([data_resampled, temp_df], ignore_index=True) 
        start_index += step
        stop_index += step
   
    return data_resampled

In [7]:
def process_data(files_folder):
    processed_data = []
    data = dataReader(files_folder)
    for i in range(len(data)):
        data_resampled = resample(data[i], target_rate)
        spectrum, xf = apply_rfft(data_resampled, target_rate)
        processed_data.append(spectrum)

    # data = dataReader(files_folder)
    # dataScaled = scalerData(data)

    # data_resampled = resample(data, target_rate)
    # spectrum, xf = apply_rfft(data_resampled, target_rate)
    # processed_data.append(spectrum)

    return np.array(processed_data)

In [24]:
teste = process_data( glob.glob('./data/normal/*.csv'))

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

data_normal = process_data( glob.glob('./data/normal/*.csv'))
data_imbalance6g= process_data( glob.glob('./data/imbalance/6g/*.csv'))
# data_imbalance10g= process_data( glob.glob('./data/imbalance/10g/*.csv'))
# data_imbalance15g= process_data( glob.glob('./data/imbalance/15g/*.csv'))
# data_imbalance20g= process_data( glob.glob('./data/imbalance/20g/*.csv'))
# data_imbalance25g= process_data( glob.glob('./data/imbalance/25g/*.csv'))
# data_imbalance30g= process_data( glob.glob('./data/imbalance/30g/*.csv'))
# data_imbalance35g= process_data( glob.glob('./data/imbalance/35g/*.csv'))

X = np.concatenate((
    data_normal,
    data_imbalance6g,
    # data_imbalance10g,
    # data_imbalance15g,
    # data_imbalance20g,
    # data_imbalance25g,
    # data_imbalance30g,
    # data_imbalance35g
    ), axis=0)

y_classes = np.concatenate((
    np.full(data_normal.shape[0], 'normal'), 
    np.full(data_imbalance6g.shape[0], 'imbalance6g'),
    # np.full(data_imbalance10g.shape[0], 'imbalance10g'),
    # np.full(data_imbalance15g.shape[0], 'imbalance15g'),
    # np.full(data_imbalance20g.shape[0], 'imbalance20g'),
    # np.full(data_imbalance25g.shape[0], 'imbalance25g'),
    # np.full(data_imbalance30g.shape[0], 'imbalance30g'),
    # np.full(data_imbalance35g.shape[0], 'imbalance35g')
     ), axis=0)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y_classes)


X_train, X_test, y_train, y_test  = train_test_split(X, y, random_state=1)

In [33]:
# import pickle
# from pathlib import Path
# pickle.dump(X, open(Path('./data/data_processed/'), 'wb'))


In [ ]:
# X = pickle.load(open('./data/data_processed/'), 'rb')

In [9]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# X_train_3d = np.expand_dims(X_train, axis=1)
# X_test_3d = np.expand_dims(X_test, axis=1)


model = Sequential()
model.add(LSTM(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=30, batch_size=int(X_train.shape[0]), validation_data=(X_test, y_test))

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy}')

Epoch 1/100
1/1 [==============================] - 10s 10s/step - loss: 1.5202 - accuracy: 0.6438 - val_loss: 1.3749 - val_accuracy: 0.7200
Epoch 2/100
1/1 [==============================] - 5s 5s/step - loss: 1.3358 - accuracy: 0.8219 - val_loss: 1.2192 - val_accuracy: 0.8000
Epoch 3/100
1/1 [==============================] - 5s 5s/step - loss: 1.1698 - accuracy: 0.8767 - val_loss: 1.0829 - val_accuracy: 0.8400
Epoch 4/100
1/1 [==============================] - 4s 4s/step - loss: 1.0212 - accuracy: 0.9041 - val_loss: 0.9642 - val_accuracy: 0.8400
Epoch 5/100
1/1 [==============================] - 5s 5s/step - loss: 0.8888 - accuracy: 0.9178 - val_loss: 0.8614 - val_accuracy: 0.8400
Epoch 6/100
1/1 [==============================] - 5s 5s/step - loss: 0.7711 - accuracy: 0.9178 - val_loss: 0.7727 - val_accuracy: 0.8400
Epoch 7/100
1/1 [==============================] - 5s 5s/step - loss: 0.6673 - accuracy: 0.9178 - val_loss: 0.6965 - val_accuracy: 0.8400
Epoch 8/100
1/1 [===============

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

model = Sequential()
model.add(GRU(100, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dense(6, activation='softmax'))
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Treinar o modelo
model.fit(X_train, y_train, epochs=30, batch_size=100, validation_data=(X_test, y_test))

# Avaliar o modelo
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Model Accuracy: {accuracy}')

Epoch 1/30
1/1 [==============================] - 14s 14s/step - loss: 2.0199 - accuracy: 0.0411 - val_loss: 1.7429 - val_accuracy: 0.2000
Epoch 2/30
1/1 [==============================] - 5s 5s/step - loss: 1.7657 - accuracy: 0.2055 - val_loss: 1.5316 - val_accuracy: 0.4000
Epoch 3/30
1/1 [==============================] - 6s 6s/step - loss: 1.5343 - accuracy: 0.3836 - val_loss: 1.3441 - val_accuracy: 0.7200
Epoch 4/30
1/1 [==============================] - 6s 6s/step - loss: 1.3284 - accuracy: 0.6301 - val_loss: 1.1816 - val_accuracy: 0.7600
Epoch 5/30
1/1 [==============================] - 5s 5s/step - loss: 1.1494 - accuracy: 0.7397 - val_loss: 1.0437 - val_accuracy: 0.8000
Epoch 6/30
1/1 [==============================] - 9s 9s/step - loss: 0.9974 - accuracy: 0.8082 - val_loss: 0.9292 - val_accuracy: 0.8000
Epoch 7/30
1/1 [==============================] - 12s 12s/step - loss: 0.8712 - accuracy: 0.7945 - val_loss: 0.8359 - val_accuracy: 0.8000
Epoch 8/30
1/1 [=====================